In [10]:

import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
import sys
sys.path.append('./lib')
from lib.cleaning import *
from lib.data_prepration import *
from lib.paralellism import *
from lib.mydoc2vec import *
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import nltk

from gensim.models import Doc2Vec
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
#!pip install scipy==1.12
SEED = 448

In [12]:
file_path = '../data/book_review.csv'
data_prep = DataPreparation(file_path)
data_prep.clean_and_preprocess(True)
data_prep.Normalize()
df=data_prep.data
df

KeyError: "['Unnamed: 0.1'] not found in axis"

In [ ]:
df.columns

In [ ]:
print(len(df))
print(df.info())
print(df.describe())

In [ ]:
df.isnull().sum()

In [ ]:
df.score.value_counts()

In [ ]:
data_prep.generate_plots()

In [ ]:
corr_matrix = data_prep.data.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
"""!python.exe -m pip install --upgrade pip 
!pip install nltk --upgrade
!pip install gensim --upgrade
!pip install scipy --upgrade"""

In [ ]:
join_func = lambda x: ', '.join(map(str, x))
data_prep.data['all']=data_prep.data['title'] + data_prep.data['description'] + data_prep.data['authors'].apply(join_func)+ data_prep.data['categories'].apply(join_func)
data_prep.data['all']

0        the hobbit or there and back againcelebrating ...
2        god's name in vain : the wrongs and rights of ...
3        the catcher in the rye [audiobook] [cd] [unabr...
4        east of edenthis sprawling and often brutal no...
5        the downing street yearsthis first volume of m...
                               ...                        
29993    harper lee's to kill a mockingbird (barron's b...
29994    the black arrow: a tale of the two rosesthis b...
29996    nicolae: left behind, volume 3continues the st...
29997    unfit for command: swift boat veterans speak o...
29998    count of monte cristothe count of monte cristo...
Name: all, Length: 23556, dtype: object

In [ ]:

#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data_prep.data['all'])]


In [ ]:

pls=Parallelism(8)
pls.set_worker()

def preprocess_document(text):
  import nltk
  from nltk.tokenize import word_tokenize
  from nltk.corpus import stopwords
  tokens = word_tokenize(text.lower())
  cleaned_tokens = [word for word in tokens if word not in stopwords.words('english')]
  return cleaned_tokens
#preprocessed_documents = pls.do_paralell(data_prep.data,'all',preprocess_document)

In [ ]:
_doc2vec = Doc2VecRecommender()
_doc2vec.train(data_prep.data)

In [ ]:
_doc2vec.model.save("doc2vec_model_hole")


In [ ]:
_doc2vec_load = Doc2VecRecommender(data_prep.data)
_doc2vec_load.load_model('doc2vec_model_hole')
search='after ice age a lion king is back'

In [ ]:
_doc2vec_load.recommend_by_text(search) 

,title,price,helpfulness,score,time,summary,text,description,authors,publisher,publisheddate,categories,ratingscount,all
9490,the call of distant mammoths: why the ice age ...,3.000000,0.8,4.0,1999-03-17,extinction in the recent past (and future?),mr. ward's book was an interesting and quick r...,to help us understand what happened during the...,[Peter D. Ward],springer science & business media,1998-01-01,[Science],2.000000,the call of distant mammoths: why the ice age ...
918,the poisonwood bible,21.296901,,4.0,2010-09-28,congolese in the congo visited by americans,"as one reviewer commented, the last part of th...",'breathtaking.' sunday times 'exquisite.' the ...,[Barbara Kingsolver],faber & faber,2002-01-01,[Fiction],280.000000,the poisonwood bible'breathtaking.' sunday tim...
16932,the vanishing hitchhiker american urban legend...,21.296901,0.961538,4.0,2000-05-10,the vanishing hitchhiker,this is a book about american urban legends an...,the book that launched america's urban legend ...,[Jan Harold Brunvand],w. w. norton & company,1997-01-01,[Fiction],5.000000,the vanishing hitchhiker american urban legend...
5486,mindstorms robotics invention system 2.0,21.296901,0.5,5.0,2002-07-02,awesome toy,this to is the best toy in the world by alot. ...,a guide to the lego mindstorms robotics invent...,"[Syngress, Mario Ferrari, Jeff Elliott, Giulio...",syngress,2002-01-01,[Computers],261.457603,mindstorms robotics invention system 2.0a guid...
10592,the norton book of women's lives,21.296901,,4.0,2013-01-14,important book of memoirs,this book has many excerpts from women's memoi...,"""this remarkable and wide-ranging collection, ...",[Phyllis Rose],w. w. norton,1995-01-01,[Biography & Autobiography],261.457603,"the norton book of women's lives""this remarkab..."
1664,one last time,21.296901,0.666667,5.0,2000-10-09,an authentic modern psychic,john edward reveals to the public the heartfel...,"a new york times, usa today, and wall street j...",[Corinne Michaels],baae publishing,2013-01-01,[Fiction],2.000000,"one last timea new york times, usa today, and ..."
20875,scleroderma: the proven therapy that can save ...,14.060000,1.0,5.0,2008-02-15,a must read for ra or sclerderma patients,i have a patient with sclerderma on this thera...,"scleroderma, which affects as many as 400,000 ...",[Henry Scammell],rowman & littlefield,2002-01-01,[Health & Fitness],261.457603,scleroderma: the proven therapy that can save ...
17262,unlocking your genetic history: a step-by-step...,21.296901,0.375,5.0,2009-06-25,great seller,book that i ordered was just as i expected. i ...,this book shows how to discover genetic predis...,[Thomas H. Shawker],thomas nelson,2004-01-01,[Reference],261.457603,unlocking your genetic history: a step-by-step...
29434,ruffian,21.296901,1.0,5.0,2005-09-29,"she was big, she was black, she was a freak",ruffian and her tragic life moves this story t...,"""a colorful story...ruffian was nothing if not...",[Jane Schwartz],ballantine books,2009-01-01,[History],8.000000,"ruffian""a colorful story...ruffian was nothing..."
21813,women who run with the wolves: myths and stori...,21.296901,,5.0,2012-04-03,important for all women to read!!!,"i'm going to keep this short and sweet, but i ...",a jungian analyst explores the feminine psyche...,[Clarissa Pinkola Estés],,1997-01-01,[Literary Criticism],38.000000,women who run with the wolves: myths and stori...
